![](images/obspy_logo_full_524x179px.png)

<div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.9) ; line-height: 100%">Breve guida a ObsPy: Python per la sismologia</div>


Questa guida trae spunto dal Tutorial di:

* Lion Krischer ([@krischer](https://github.com/krischer))
* Tobias Megies ([@megies](https://github.com/megies))

Seismo-Live: http://seismo-live.org

---

**Questa guida si propone di illustrare le funzioni e le classi di ObsPy**

In sismologia possiamo distinguere 3 tipi fondamentali di dati e ciascuno di essi viene letto da una funzione in ObsPy:
* `obspy.read()`: legge le forme d'onda dagli oggetti `obspy.Stream` e `obspy.Trace`.
* `obspy.read_inventory()`: legge le informazioni sulle stazioni dagli oggetti `obspy.Inventory`.
* `obspy.read_events()`: legge gli eventi dagli oggetti `obspy.Catalog`.

Il formato specifico di ciascuno di questi tipi di dato viene determinato automaticamente da ObsPy, ciascuna funzione supporta la lettura da URL, vari formati di compressione, file in memoria e altre fonti. Molti formati di file sono disponibili anche in scrittura. Gli oggetti risultanti consentono la manipolazione dei dati in vari modi.

Uno degli obiettivi principali di ObsPy è quello di evitare che i ricercatori si debbano preoccupare del formato in cui i loro dati sono scritti per concentrarsi esclusivamente sul compito da svolgere.

In [ ]:
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 16, 8

## FDSN/SEED Identificatori

In accordo allo [standard SEED](http://www.fdsn.org/media/_s/publications/SEEDManual_V2.4.pdf), la nomenclatura che segue viene usata per identificare le stazioni sismiche:

* **Network identifier**: Identifica la rete/proprietà dei dati. È assegnata dalla FDSN ed è quindi unica.
* **Station identifier**: Identifica la stazione all'interno della rete. *NON È UNICA. Va sempre usata insieme al codice della rete!*
* **Location identifer**: Identifica differenti stream di dati per la stessa stazione. Normalmente usato per separare più strumenti alla stessa stazione.
* **Channel identifier**: Identificatore dei canali. Abbiamo tre codici per i canali: 1) Banda e frequenza di campionamento approssimativa, 2) Il tipo di strumento, 3) L'orientazione

A questo punto abbiamo l'identificazione completa del modulo **NET.STA.LOC.CHA**, ad es. **IV.PII.00.BHZ.** *Rete IV (INGV), Stazione PII (Pisa), Location 00, Canale B (broad band) H (High Gain Seismometer) Z (componente verticale)*


---

In sismologia generalmente facciamo distinzione tra 3 diversi tipi di dato:

1. **Waveform Data** - Dati delle forme d'onda come serie temporali.
2. **Station Data** - Informazioni sulle stazioni, coordinate geografiche, risposta dello strumento.
3. **Event Data** - Informazioni sui terremoti ed altre sorgenti sismiche.


## Waveform Data

![stream](images/Stream_Trace.svg)

Esistono una miriade di formati di dati delle forme d'onda, ma in Europa e negli Stati Uniti dominano due formati: **MiniSEED** e **SAC**. Qui vedremo solo i **MiniSEED**


### MiniSEED

* È ciò che ottieni dai datacenter e solitamente anche ciò che memorizzano, si tratta dei dati originali.
* Molto utile come formato di streaming e archiviazione.
* È possibile memorizzare interi e float in singola/doppia precisione. 
* I dati interi (ad esempio i count da un digitalizzatore) sono fortemente compressi: un fattore di 3-5 a seconda dei dati.
* Può gestire gap e sovrapposizioni.
* Più componenti per ciascun file.
* Contiene solo i parametri veramente necessari e alcune informazioni per gli gestori di rete e i fornitori di dati.

In [ ]:
# Per usare ObsPy come prima cosa bisogna sempre importarlo
import obspy

# ObsPy individua SEMPRE il formato del file
# (indipendentemente dal formato dei dati) restituisce l'oggetto Stream.
st = obspy.read("INGV/PieveTorina.mseed")

# Stampare un oggetto di solito si traduce in qualche tipo di
# stringa informativa.
print(st)

In [ ]:
# Le informazioni contenute nella prima traccia vengono 
# visualizzate con il seguente comando (.stats è un attributo)
print(st[0].stats)

In [ ]:
# Usa il metodo .plot() per una rapida visualizzazione 
# del contenuto dello stream.
st.plot()

In [ ]:
# Qui vedremo brevemente come lavorare con gli oggetti Stream/Trace.

# La maggior parte delle operazioni modifica gli oggetti esistenti. 
# Questo viene fatto per motivi di prestazioni.

# Creiamo quindi una copia dello stream in modo che le esecuzioni multiple di questa cella di notebook 
# non vadano a modificare i dati originali.
st2 = st.copy()

# Per usare solo parte dello stream, usare la funzione select(). In questo caso solo la componente Z
print(st2.select(component="Z"))

# Gli oggetti Stream si comportano come una lista di oggetti Trace. 
# In questo caso tr è la prima traccia dello stream st2
tr = st2[0]

# Il plot funziona anche per le singole tracce.
tr.plot()

# Alcune azioni di processamento dati. Notate che queste operazioni modificano 
# il file esistente. 
tr.detrend("linear")
tr.taper(type="hann", max_percentage=0.05)
tr.filter("lowpass", freq=2.0, corners=4)

# Ripetere il Plot.
tr.plot()

In [ ]:
# È possibile scrivere il file semplicemente specificando il formato.
st.write("temp.mseed", format="mseed")

## Station Data

![inv](images/Inventory.svg)

I dati della stazione contengono informazioni sull'organizzazione che raccoglie i dati, le informazioni geografiche e la risposta dello strumento. Principalmente disponibili in tre formati:

* `(dataless)SEED`: formato binario completo ma molto complesso. Ancora usato molto, ad es. per il protocollo Arclink
* `RESP`: Un sottoinsieme ristretto di SEED. Basato su ASCII. Contiene **SOLO** la risposta dello strumento. 
* `StationXML`: Essenzialmente come SEED ma più pulito e basato su XML. È il formato più moderno e ciò che i data center oggi forniscono. **Se puoi usa questo.**


ObsPy può lavorare con tutti allo stesso modo, ma in questa guida ci concentreremo su StationXML.

Questi sono file XML:

In [ ]:
!head INGV/stazioni.xml

In [ ]:
import obspy

# Usa la funzione read_inventory per aprirli. Questa funzione
# restituirà un oggetti Inventory.
inv = obspy.read_inventory("INGV/stazioni.xml")
print(inv)

Come puoi vedere possono contenere un numero arbitrario di reti, stazioni e canali.

In [ ]:
# Con ObsPy è possibile metterli su una mappa.
inv.plot(projection="local");

In [ ]:
# È possibile anche estrarre le coordinate dei singoli canali. 
# Questa funzione accetta anche argomenti datetime per estrarre 
# informazioni in diversi momenti nel tempo.
inv.get_coordinates("IV.PII..HHZ")

In [ ]:
# E naturalmente può essere scritta di nuovo, anche in uno stato modificato.
inv.select(channel="BHZ").write("temp.xml", format="stationxml")

## Event Data

![events](./images/Event.svg)

I dati degli eventi sono spesso forniti in formati molto semplici come NDK o il formato CMTSOLUTION utilizzato da molti solutori di forme d'onda:

In [ ]:
!head ./INGV/tirreno_eq.xml

Datacenters forniscono file **QuakeML**, che sono sorprendentemente complessi per quanto riguarda la loro struttura ma possono contenere relazioni dettagliate tra parti di dati.

In [ ]:
# Leggi i file QuakeML con la funzione read_events().
cat = obspy.read_events("INGV/tirreno_eq.xml")
print(cat)

In [ ]:
print(cat[0])

In [ ]:
cat.plot(projection="ortho");

In [ ]:
# Ancora ua volta possono essere riscritti con la funzione write().
cat.write("temp_quake.xml", format="quakeml")

Per mostrare altre cose, ho aggiunto un file contenente tutti gli eventi che l'INGV ha localizzato in un raggio di 50 km intorno a Lucca a partire dal 2010 con magnitudo uguale a 2 o superiore.

In [ ]:
import obspy

cat = obspy.read_events("./INGV/lucca.xml")

print(cat)

In [ ]:
cat.plot(projection="local", resolution="i");

In [ ]:
cat.filter("magnitude > 3").plot(projection="local", resolution="i");